In [2]:
import warnings

warnings.filterwarnings('ignore', message= 'Series.__getitem__')

In [3]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv('dataset_topics_60k.csv')
df.dtypes

Unnamed: 0            int64
0                    object
1                    object
2                    object
source               object
hash                 object
topic1              float64
topic2              float64
topic3              float64
topic4              float64
topic5              float64
topic6              float64
topic7              float64
topic8              float64
topic9              float64
topic10             float64
topic11             float64
topic12             float64
topic13             float64
topic14             float64
topic15             float64
best topic           object
best probability    float64
dtype: object

In [5]:
# Function to create a string instance based on specific columns in a DataFrame
def combine_columns_to_string(index):
    # Ensuring the index is non-negative
    assert index >= 0, 'Index cannot be a negative integer'

    # Retrieving the row at the specified index
    selected_row = df.iloc[index, :]

    # Combining columns 0, 1, and 2 into a single string
    # making the poem seem together at similar manner
    combined_string = str(selected_row[1]) +' ' +str(selected_row[2]) + ' '+str(selected_row[3])

    return combined_string

# Applying the function to each row in the df
document = [combine_columns_to_string(i) for i in range(len(df.iloc[:, 2]))]
print(len(document))
document[:10]

26327


["visiting the graves stronger the october wind at my grandparents'",
 'profound blue of night  the resin and salt of pines so far from the sea',
 'scattered in the ditch  like tiny scraps of blue sky bits of plastic bag',
 'the smell of her hands on the neck of the bottle drinking greedily',
 "christmas services a cellular phone rings out handel's messiah",
 "gazing at the moon on a still summer's evening feast for mosquitoes",
 'my tea gets colder and the madeleine just sinks memory betrays',
 'small green waves crashing against a porcelain rim morning tea tempest',
 'red poppies growing between rows of white tombstones as in remembrance',
 'in front of bronze doors they huddle against the cold the newly homeless']

In [6]:
#cleaning the document
document = [string.replace('\'', '') for string in document]
document[:5]

['visiting the graves stronger the october wind at my grandparents',
 'profound blue of night  the resin and salt of pines so far from the sea',
 'scattered in the ditch  like tiny scraps of blue sky bits of plastic bag',
 'the smell of her hands on the neck of the bottle drinking greedily',
 'christmas services a cellular phone rings out handels messiah']

In [7]:
import nltk

nltk.download('cmudict')

cmu_dict =nltk.corpus.cmudict.dict()
words_not_in_cmu = []
len(cmu_dict)

[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\OMEN\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


123455

In [8]:
def num_syllables_word(word):
    if word == '<mask>':
        return -1
    try:
        # from: https://datascience.stackexchange.com/questions/23376/how-to-get-the-number-of-syllables-in-a-word
        return [len(list(y for y in x if y[-1].isdigit())) for x in cmu_dict[word.lower()]][0]
    except:
        return -1
    
def n_syllables(sentence):
    num_syllabes = 0
    for word in sentence.split():
        num_s = num_syllables_word(word)
        if num_s != -1:
            num_syllabes += num_syllables_word(word)
    return num_syllabes

In [9]:
n_syllables('I am a deep learning student'), n_syllables('student')

(8, 2)

In [10]:
#getting all the topics for each document poem
all_topics = df['best topic'].values.astype(str)
type(all_topics[0])

numpy.str_

In [11]:
#arranging the masks randomly for each words and embedding the syllable count for each stanza?
def random_masking(line):
    
    indv_words = line.split()
    for i in range(len(indv_words)):
        rand = np.random.uniform(0, 1)
        
        if rand>0.6:
            indv_words[i] = '<*>'
    line = ' '.join(indv_words)
    return line

In [12]:
random_masking('There are many potatoes in the sack')

'There <*> many <*> in the sack'

In [13]:
def mask_entire_line(full_line):
    masked_line = []
    masked_part_line = ''
    
    for part_line in full_line:
        masked_part_line = random_masking(part_line)
        masked_line.append(masked_part_line)
        
    return masked_line

In [14]:
#every time, the masking is different to make it random while training
mask_entire_line(['Here we go now far along', 'all the way into the abyss', 'and keep it common that we are the best'])

['<*> <*> go <*> <*> along',
 'all <*> way into <*> abyss',
 'and keep it <*> that <*> <*> <*> best']

In [15]:
#start and end token
ST = '<s>'#Start Token
ET = '</s>'#End Token

In [16]:
# creating masked poems and original poems for the comparison
# this one just to see what really happens in it
def mask_df_demo(df):
    masked_poems = []
    org_poems = []
    
    for i, comb in enumerate(df.iterrows()):
        index, row = comb
        poem_topic = all_topics[i]
        
        mask_1 = random_masking(row[1])
        mask_2 = random_masking(row[2])
        mask_3 = random_masking(row[3])
        
        syllable_1 = str(n_syllables(row[1]))
        syllable_2 = str(n_syllables(row[2]))
        syllable_3 = str(n_syllables(row[3]))
        
        masked_poem_1 = ' '.join([poem_topic, ST, mask_1, syllable_1, ET])
        masked_poem_2 = ' '.join([ST, mask_2, syllable_2, ET])
        masked_poem_3 = ' '.join([ST, mask_3, syllable_3, ET])
        
        org_poem_1 = ' '.join([poem_topic, ST, row[1], syllable_1, ET])
        org_poem_2 = ' '.join([ST, row[2], syllable_2, ET])
        org_poem_3 = ' '.join([ST, row[3], syllable_3, ET])
        
        overall_masked_poem = ' '.join([masked_poem_1,masked_poem_2, masked_poem_3])
        overall_org_poem = ' '.join([org_poem_1,org_poem_2, org_poem_3])
        print(overall_masked_poem)
        print(overall_org_poem)    

In [17]:
mask_df_demo(df.head(1))

topic12 <s> <*> the graves 5 </s> <s> <*> the october wind 7 </s> <s> at <*> <*> 5 </s>
topic12 <s> visiting the graves 5 </s> <s> stronger the october wind 7 </s> <s> at my grandparents' 5 </s>


In [93]:
# creating masked poems and original poems for the comparison
def mask_df(df):
    masked_poems = []
    org_poems = []
    
    for i, comb in enumerate(df.iterrows()):
        index, row = comb
        poem_topic = all_topics[i]
        
        mask_1 = random_masking(row[1])
        mask_2 = random_masking(row[2])
        mask_3 = random_masking(row[3])
        
        syllable_1 = str(n_syllables(row[1]))
        syllable_2 = str(n_syllables(row[2]))
        syllable_3 = str(n_syllables(row[3]))
        
        masked_poem_1 = ' '.join([poem_topic, ST, mask_1, syllable_1, ET])
        masked_poem_2 = ' '.join([ST, mask_2, syllable_2, ET])
        masked_poem_3 = ' '.join([ST, mask_3, syllable_3, ET])
        
        org_poem_1 = ' '.join([poem_topic, ST, row[1], syllable_1, ET])
        org_poem_2 = ' '.join([ST, row[2], syllable_2, ET])
        org_poem_3 = ' '.join([ST, row[3], syllable_3, ET])
        
        overall_masked_poem = ' '.join([masked_poem_1,masked_poem_2, masked_poem_3])
        overall_org_poem = ' '.join([org_poem_1,org_poem_2, org_poem_3])
        
        masked_poems.append(overall_masked_poem)
        org_poems.append(overall_org_poem)
        
        
    assert(len(masked_poems)==len(org_poems))
    return masked_poems, org_poems

In [94]:
X, Y = mask_df(df)

In [89]:
len(X), len(Y)

(26327, 26327)

In [92]:
len(X[0]), len(Y[0])

(94, 106)

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=True)
len(X_train), len(y_test)

(21061, 5266)

In [63]:
import transformers
from transformers import AlbertTokenizer, AlbertModel, AlbertForSequenceClassification

In [23]:
import torch
import torch.nn as nn
from tqdm.auto import tqdm

In [24]:
torch.cuda.empty_cache()

In [25]:
import sentencepiece

In [28]:
HF_HUB_DISABLE_SYMLINKS_WARNING = 1

In [64]:
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')#alBERTa tokenizer
model = AlbertForSequenceClassification.from_pretrained('albert-base-v2')#alBERTa model which is pretrained

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [85]:
type(X_train)

list

In [86]:
len(X_train[0]), len(y_train[0])

(97, 112)

In [83]:
%%time

X_train_tokenized = [tokenizer(xtr, return_tensors="pt", padding=True) for xtr in X_train]
X_test_tokenized = [tokenizer(xte, return_tensors="pt", padding=True) for xte in X_test]
y_train_tokenized = [tokenizer(ytr, return_tensors="pt", padding=True) for ytr in y_train]
y_test_tokenized = [tokenizer(yte, return_tensors="pt", padding=True) for yte in y_test]
print(X_train_tokenized[0]["input_ids"].shape, y_train_tokenized[0]["input_ids"].shape)

torch.Size([1, 61]) torch.Size([1, 46])
CPU times: total: 15.1 s
Wall time: 15.3 s


In [66]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [67]:
model.to(device)
model

AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768,

In [36]:
model.to(device)
model

AlbertModel(
  (embeddings): AlbertEmbeddings(
    (word_embeddings): Embedding(30000, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 128)
    (token_type_embeddings): Embedding(2, 128)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0, inplace=False)
  )
  (encoder): AlbertTransformer(
    (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
    (albert_layer_groups): ModuleList(
      (0): AlbertLayerGroup(
        (albert_layers): ModuleList(
          (0): AlbertLayer(
            (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (attention): AlbertAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (attention_dropout): Dropout(p=0, inplace=False)
      

In [68]:
print(X_train_tokenized[0], y_train_tokenized[0])
len(X_train_tokenized), len(X_test_tokenized)

{'input_ids': tensor([[   2, 8303,  240,   13,    1,   18,    1,   13,    1, 2483,    1,   13,
            1, 2483,    1,  331,   13,    1,  118,   18,    1,   13,    1,   18,
            1,   13,    1, 2483,    1,   59,  114, 2985,   17,   14,  453,   13,
            1,  118,   18,    1,   13,    1,   18,    1,  126,   13,    1, 2483,
            1,   13,    1, 2483,    1, 7837,  331,   13,    1,  118,   18,    1,
            3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])} {'input_ids': tensor([[   2, 8303,  240,   13,    1,   18,    1, 2361, 5664,  331,   13,    1,
      

(21061, 5266)

In [69]:
model.parameters()

<generator object Module.parameters at 0x0000023C55802F80>

In [70]:
optimizer = torch.optim.AdamW(model.parameters(), lr = 4e-4)
optimizer

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0004
    maximize: False
    weight_decay: 0.01
)

In [71]:
epochs = 3

In [72]:
training_steps = epochs * len(X_train)
test_steps = epochs * len(X_test)
training_steps, test_steps

(63183, 15798)

In [52]:
help(transformers.get_scheduler)

Help on function get_scheduler in module transformers.optimization:

get_scheduler(name: Union[str, transformers.trainer_utils.SchedulerType], optimizer: torch.optim.optimizer.Optimizer, num_warmup_steps: Optional[int] = None, num_training_steps: Optional[int] = None)
    Unified API to get any scheduler from its name.
    
    Args:
        name (`str` or `SchedulerType`):
            The name of the scheduler to use.
        optimizer (`torch.optim.Optimizer`):
            The optimizer that will be used during training.
        num_warmup_steps (`int`, *optional*):
            The number of warmup steps to do. This is not required by all schedulers (hence the argument being
            optional), the function will raise an error if it's unset and the scheduler type requires it.
        num_training_steps (`int``, *optional*):
            The number of training steps to do. This is not required by all schedulers (hence the argument being
            optional), the function will raise

In [73]:
lr_scheduler = transformers.get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=training_steps
)
lr_scheduler

In [74]:
criterion = nn.CrossEntropyLoss()
criterion

CrossEntropyLoss()

In [81]:
X_train_tokenized[3]["input_ids"].shape, y_train_tokenized[3]["input_ids"].shape

(torch.Size([1, 66]), torch.Size([1, 49]))

In [80]:
model(X_train_tokenized[3]["input_ids"].to(device), y_train_tokenized[3]["input_ids"].to(device))

RuntimeError: The size of tensor a (66) must match the size of tensor b (49) at non-singleton dimension 3

In [77]:
# logger for train and validation loss
training_loss_logger = []
test_loss_logger = []

train_3k_losses = []
test_3k_losses = []


for epoch in range(1):
    #torch.cuda.empty_cache()
    # train
    model.train()
    train_loss = 0
    train_done = 0
    print('Train started')
    for i in range(len(X_train_tokenized[:6001])):
        if X_train_tokenized[i]["input_ids"].shape[1] == y_train_tokenized[i]["input_ids"].shape[1]:
            
            ## training the dataset taking X[i] and Y[i] of training sets
            inp = X_train_tokenized[i]["input_ids"].to(device)
            label = y_train_tokenized[i]["input_ids"].to(device)
            print(inp, label)
            model_outputs = model(inp, label)
            print(model_outputs)
            loss = model_outputs.loss
            loss.backward()
            
            # manipulating steps and then resetting gradients with zero_grad at the end
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            
            #managing the loss back to appropriate format
            loss = loss.cpu().detach().numpy()
            
            train_loss = train_loss + loss
            train_done = train_done + 1
            break
            if i%3000==0:
                train_3k_losses.append([i, train_loss/train_done])
                print(f'Epoch: {epoch}, Train: {i} / {len(X_train_tokenized)}, loss = {train_loss/train_done}, Remaining: {len(X_train_tokenized)-i}')
    
    
    train_skipped = len(X_test_tokenized) - train_done
    train_loss = train_loss/train_done  
    print(f"epoch: {epoch+1}, train loss: {train_loss}, Skipped = {train_skipped}")

    
    
    # Test
    model.eval()
    test_loss = 0
    test_done = 0
    for i in range(len(X_test_tokenized)):
        if X_test_tokenized[i]["input_ids"].shape[1] == y_test_tokenized[i]["input_ids"].shape[1]:
            
            ## testing the dataset taking X[i] and y[i] of test datasets
            inp = X_test_tokenized[i]["input_ids"].to(device)
            label = y_test_tokenized[i]["input_ids"].to(device)
            
            with torch.no_grad():
                outputs = model(inp, labels = label)
                
            loss = outputs.loss
            
            #managing the format of loss for alignment
            loss = loss.cpu().detach().numpy()
            
            test_loss = validate_loss + loss
            test_done = validate_done+1
            break
            if i%3000==0:
                test_3k_losses.append([i, test_loss/test_done])
                print(f'Epoch: {epoch}, Test: {i} / {len(X_test_tokenized)}, loss = {test_loss/test_done}, Remaining: {len(X_test_tokenized)-i}')
            
            
    test_skipped = len(X_test_tokenized) - test_done
    test_loss = test_loss/test_done  
    
    print(f"epoch: {epoch+1}, validate loss: {test_loss}")
    training_loss_logger.append(train_loss)
    validation_loss_logger.append(test_loss)
    break

Train started
tensor([[    2,  8303,   264,    13,     1,    18,     1,   100, 23708,   331,
            13,     1,   118,    18,     1,    13,     1,    18,     1,  4041,
            31,    22,    43,    44,    86,  1700,    32,   453,    13,     1,
           118,    18,     1,    13,     1,    18,     1,  1380,    21,   865,
            20,    55,   331,    13,     1,   118,    18,     1,     3]],
       device='cuda:0') tensor([[    2,  8303,   264,    13,     1,    18,     1,   100, 23708,   331,
            13,     1,   118,    18,     1,    13,     1,    18,     1,  4041,
            31,    22,    43,    44,    86,  1700,    32,   453,    13,     1,
           118,    18,     1,    13,     1,    18,     1,  1380,    21,   865,
            20,    55,   331,    13,     1,   118,    18,     1,     3]],
       device='cuda:0')
SequenceClassifierOutput(loss=None, logits=tensor([[nan, nan]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


AttributeError: 'NoneType' object has no attribute 'backward'